In [26]:
cd /content/drive/My Drive/StackQuestioner

/content/drive/My Drive/StackQuestioner


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re

In [28]:
dataset=pd.read_csv('dataset.csv',delimiter=';')

In [29]:
dataset.head()

,title,id_stack,tags,views,score,done,label
0,Using entries from other kivy classes,61881920,"['python', 'python-3.x', 'kivy']",12,0,False,python
1,Package python software with pylucene dependency,61896481,"['python', 'docker', 'pip', 'dependencies', 'p...",7,1,False,python
2,Extracting time with regex from a string,61894597,"['python', 'regex']",29,3,False,python
3,How do I add specific headers before each form...,61896721,"['python', 'django']",4,0,False,python
4,Barplot from a dataframe using a column to set...,61896506,"['python', 'pandas', 'bar-chart', 'seaborn', '...",12,0,True,python


In [30]:
dataset.shape

(75334, 7)

In [31]:
title_features=dataset['title']

In [32]:
title_features=list(title_features)

In [33]:
label=dataset['label']

In [34]:
label=list(label)

In [35]:
set(label)

{'R', 'java', 'javascript', 'php', 'python'}

In [36]:
text_data=[]

In [37]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
for line in title_features:
  sent=re.sub('[^a-zA-Z]',' ',line)
  sent_lower=sent.lower()
  words_split=sent_lower.split(' ')
  words=[x for x in words_split if x not in stopwords.words('english')]
  sentence=' '.join(words)
  text_data.append(sentence)


In [39]:
from sklearn.preprocessing import LabelEncoder

In [40]:
le=LabelEncoder()

In [41]:
label=le.fit_transform(label)

In [42]:
le.classes_

array(['R', 'java', 'javascript', 'php', 'python'], dtype='<U10')

In [43]:
from keras.utils.np_utils import to_categorical

In [44]:
label=to_categorical(label)

In [45]:
label

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [46]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [47]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [48]:
tokenizer=Tokenizer()

In [49]:
tokenizer.fit_on_texts(text_data)

In [50]:
import pickle
pickle.dump(tokenizer,open('tokenizer.pkl','wb+'),protocol=pickle.HIGHEST_PROTOCOL)

In [51]:
sequences_text=tokenizer.texts_to_sequences(text_data)

In [52]:
sequences_text

[[1, 889, 862, 332],
 [97, 2, 1726, 12032, 591],
 [712, 40, 179, 18],
 [28, 69, 802, 64, 82, 5763],
 [1440, 48, 1, 23, 51, 260, 1003],
 [2, 1922, 69, 9, 15, 1661, 163],
 [117, 104, 91, 74, 843, 237, 13, 4815, 46],
 [19, 11, 352, 387, 35, 154, 27, 14, 35, 17],
 [2, 6435, 4816, 15, 2752, 244, 15, 32, 8890],
 [1688, 1662, 1155, 53],
 [659, 315, 2, 3779, 2340, 69, 151],
 [2, 6, 53, 341, 276, 533],
 [8891, 910, 1, 2, 6435, 3451],
 [26, 2408, 34, 87],
 [3780, 630, 12033, 1208, 424, 2753, 3298, 66, 2],
 [75, 2, 869, 103, 283, 713],
 [104, 92, 32, 1759, 721, 1171, 288, 16, 104, 1759, 2684, 50],
 [434, 137, 73, 3590, 314],
 [439, 890, 122, 19, 239, 18, 96, 47, 1959],
 [110, 669, 361, 1, 421],
 [424, 81, 18, 1, 2, 179],
 [473, 4001, 100, 1880, 105, 253, 534, 56, 20, 70, 1881, 6436, 983],
 [2072, 834, 139, 89, 186, 388, 208],
 [151, 12034, 894, 878, 2685, 143, 88],
 [41, 65, 15, 15, 69, 506],
 [89, 3781, 595, 17, 5764],
 [12035, 141, 139, 302],
 [1039, 1440, 3591],
 [33, 11, 588, 446, 1109, 80, 5

In [53]:
max_lens=[]
for x in sequences_text:
  max_lens.append(len(x))

In [54]:
max(max_lens)

25

In [55]:
del(max_lens)

In [56]:
pad_sequen_text=pad_sequences(sequences_text,maxlen=26,padding='post')

In [57]:
pad_sequen_text.shape

(75334, 26)

In [58]:
pad_sequen_text=pad_sequen_text.reshape(pad_sequen_text.shape[0],pad_sequen_text.shape[1],1)

In [59]:
pad_sequen_text.shape

(75334, 26, 1)

In [60]:
label.shape

(75334, 5)

In [61]:
size=len(tokenizer.word_index)

In [62]:
size

25518

In [63]:
from sklearn.model_selection import train_test_split
feature_train,feature_test,label_train,label_test=train_test_split(pad_sequen_text,label,test_size=0.1,random_state=7)

In [64]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Embedding,Dense,Dropout

In [65]:
model=Sequential()
model.add(Embedding(size+1,64,input_length=26))
model.add(LSTM(200))
model.add(Dropout(0.2))
model.add(Dense(1024,activation='relu'))
model.add(Dense(5,activation='softmax'))


In [66]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 26, 64)            1633216   
_________________________________________________________________
lstm (LSTM)                  (None, 200)               212000    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              205824    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 5125      
Total params: 2,056,165
Trainable params: 2,056,165
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics='accuracy')

In [68]:
model.fit(feature_train,label_train,epochs=10)

Epoch 1/10
2119/2119 [==============================] - 132s 62ms/step - loss: 0.7758 - accuracy: 0.6899
Epoch 2/10
2119/2119 [==============================] - 129s 61ms/step - loss: 0.4488 - accuracy: 0.8459
Epoch 3/10
2119/2119 [==============================] - 132s 62ms/step - loss: 0.3629 - accuracy: 0.8784
Epoch 4/10
2119/2119 [==============================] - 137s 64ms/step - loss: 0.3097 - accuracy: 0.8955
Epoch 5/10
2119/2119 [==============================] - 132s 62ms/step - loss: 0.2698 - accuracy: 0.9066
Epoch 6/10
2119/2119 [==============================] - 128s 61ms/step - loss: 0.2431 - accuracy: 0.9143
Epoch 7/10
2119/2119 [==============================] - 129s 61ms/step - loss: 0.2170 - accuracy: 0.9215
Epoch 8/10
2119/2119 [==============================] - 136s 64ms/step - loss: 0.1946 - accuracy: 0.9295
Epoch 9/10
2119/2119 [==============================] - 131s 62ms/step - loss: 0.1783 - accuracy: 0.9351
Epoch 10/10
2119/2119 [==============================] 

In [69]:
model.fit(feature_train,label_train,epochs=5)

Epoch 1/5
2119/2119 [==============================] - 131s 62ms/step - loss: 0.1447 - accuracy: 0.9466
Epoch 2/5
2119/2119 [==============================] - 136s 64ms/step - loss: 0.1305 - accuracy: 0.9518
Epoch 3/5
2119/2119 [==============================] - 130s 61ms/step - loss: 0.1195 - accuracy: 0.9557
Epoch 4/5
2119/2119 [==============================] - 129s 61ms/step - loss: 0.1081 - accuracy: 0.9604
Epoch 5/5
2119/2119 [==============================] - 137s 65ms/step - loss: 0.0986 - accuracy: 0.9636


In [79]:
model.fit(feature_train,label_train,epochs=5)

Epoch 1/5
2119/2119 [==============================] - 131s 62ms/step - loss: 0.0897 - accuracy: 0.9665
Epoch 2/5
2119/2119 [==============================] - 129s 61ms/step - loss: 0.0804 - accuracy: 0.9708
Epoch 3/5
2119/2119 [==============================] - 132s 62ms/step - loss: 0.0735 - accuracy: 0.9729
Epoch 4/5
2119/2119 [==============================] - 129s 61ms/step - loss: 0.0655 - accuracy: 0.9760
Epoch 5/5
2119/2119 [==============================] - 129s 61ms/step - loss: 0.0581 - accuracy: 0.9792


In [80]:
model.evaluate(feature_test,label_test)

236/236 [==============================] - 4s 17ms/step - loss: 1.3927 - accuracy: 0.7964


[1.3927037715911865, 0.7963896989822388]

In [81]:
text=["Creating list in PHP"]

In [82]:
text=tokenizer.texts_to_sequences(text)

In [83]:
text

[[119, 15, 5]]

In [84]:
tokenizer.sequences_to_texts([[119,15,2]])

['creating list python']

In [85]:
text=pad_sequences(text,maxlen=26,padding='post')

In [86]:
text

array([[119,  15,   5,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]],
      dtype=int32)

In [87]:
model.predict(text)

array([[6.2809005e-07, 5.1036468e-06, 1.2292959e-03, 9.9876404e-01,
        9.8615874e-07]], dtype=float32)

In [88]:
model.save('model.h5')